**Task 07: Querying RDF(s)**

In [26]:
# !pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2023-2024/master/Assignment4/course_materials"

First let's read the RDF file

In [27]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=N516c36a866d8467bb71a399f4d01157a (<class 'rdflib.graph.Graph'>)>

In [90]:
ns = Namespace("http://somewhere#")
for s,p,o in g.triples((None, None, ns.Animal)):
  print(s, p, o)

http://somewhere#RockySmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Animal


**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [62]:
# TO DO
from rdflib.plugins.sparql import prepareQuery

livingThing_query = prepareQuery('''
SELECT ?subClase
WHERE {
    ?subClase rdfs:subClassOf ns:LivingThing
}                      
''',
initNs = { "rdfs": RDFS, "ns": ns}
)
# Visualize the results

for r in g.query(livingThing_query):
  print(r.subClase)

http://somewhere#Person
http://somewhere#Animal


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [72]:
# TO DO
from rdflib.plugins.sparql import prepareQuery

person_query = prepareQuery('''
SELECT ?person
WHERE {
    ?person rdf:type ns:Person
}                      
''',
initNs = { "rdf": RDF, "rdfs": RDFS, "ns": ns}
)
# Visualize the results

for r in g.query(person_query):
  print(r.person)

http://somewhere#SaraJones
http://somewhere#JohnSmith


**TASK 7.3: List all individuals of "Person" or "Animal" and all their properties including their class with RDFLib and SPARQL. You do not need to list the individuals of the subclasses of person**


In [100]:
# TO DO
from rdflib.plugins.sparql import prepareQuery

query_3 = prepareQuery('''
SELECT ?subject ?predicate ?object
WHERE {
     ?subject ?predicate ?object .
     ?subject rdf:type ?type .
FILTER (
     ?type = ns:Animal || ?type = ns:Person
)
}
''',
initNs = { "rdf": RDF, "rdfs": RDFS, "ns": ns}
)

# Visualize the results

for r in g.query(query_3):
  print(r.subject, r.predicate, r.object)

http://somewhere#RockySmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Animal
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/Given Rocky
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/FN Rocky Smith
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#RockySmith http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
http://somewhere#RockySmith http://xmlns.com/foaf/0.1/knows http://somewhere#SaraJones
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere

**TASK 7.4:  List the name of the persons who know Rocky**

In [167]:
from rdflib import FOAF
ns = Namespace("http://somewhere#")
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

#Prueba preliminar
for s,p,o in g.triples((None, FOAF.knows, None)):
  print(s, p, o)

http://somewhere#RockySmith http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
http://somewhere#RockySmith http://xmlns.com/foaf/0.1/knows http://somewhere#SaraJones
http://somewhere#JaneSmith http://xmlns.com/foaf/0.1/knows http://somewhere#SaraJones
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith
http://somewhere#JaneSmith http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith


In [161]:
# TO DO
from rdflib.plugins.sparql import prepareQuery
from rdflib import FOAF
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0")

query_4 = prepareQuery('''
SELECT ?Rocky_URI  ?amigo_URI ?amigo_name
WHERE {
     ?Rocky_URI <http://www.w3.org/2001/vcard-rdf/3.0/Given>  ?given_name . 
     ?amigo_URI foaf:knows ?Rocky_URI .
     ?amigo_URI <http://www.w3.org/2001/vcard-rdf/3.0/Given>  ?amigo_name . 
                                   

     FILTER(?given_name = "Rocky")
}
''',
initNs = { "foaf": FOAF , "ns" : ns, "vcard": VCARD}
)

# Visualize the results

for r in g.query(query_4):
     print(r.amigo_name)

Sara
Jane


**Task 7.5: List the entities who know at least two other entities in the graph**

In [238]:
# TO DO
g.add((ns.A , FOAF.knows, ns.B))
from rdflib.plugins.sparql import prepareQuery
from rdflib import FOAF
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0")

query_5 = prepareQuery('''
SELECT ?entity ?relatedEntity
WHERE {
      ?entity foaf:knows ?relatedEntity .

}
GROUP BY ?entity 
HAVING (COUNT(DISTINCT ?relatedEntity) >= 2)
    
''',
initNs = { "foaf": FOAF , "ns" : ns, "vcard": VCARD}
)

# Visualize the results

for r in g.query(query_5):
     print(r.entity)

http://somewhere#RockySmith
http://somewhere#SaraJones
http://somewhere#JaneSmith
